matrix transformation only for perfect file (with path 0)

In [75]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize

# pymatgen libraries
from pymatgen.core.structure import Structure

# import function from .py file
from get_weirdos_ver3_new_cleaned import get_dx_dz_init, get_sum_weirdos_Li_var
# from get_weirdos import get_sum_weirdos_Li
# from modify_dx_dz import create_file_name, change_dx_dz
# from get_weirdos import get_dx1_dx2_dz_init, get_sum_weirdos_Li

In [76]:
# from get_weirdos_ver3_new_cleaned import varying_radius_vs_sumweirdosLi, plot_varying_radius_vs_sumweirdosLi

In [77]:
direc = os.getcwd() # get current working directory

# # filefile_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_type2.cif"_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"  # Replace with your file path
  # Replace with your file path
# file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type5_ver1_a"
file_perfect_poscar_48n24_wo_cif_array = ["Li6PS5Cl_type5_trial"]
# file_perfect_poscar_48n24_wo_cif_array = ["Li6PS5Cl_type5_ver1_a_less_opti", "Li6PS5Cl_type5_ver1_b_less_opti", "Li6PS5Cl_type5_ver2_a_less_opti", "Li6PS5Cl_type5_ver2_b_less_opti", 
#                                           "Li6PS5Cl_type5_ver3_a_less_opti", "Li6PS5Cl_type5_ver3_b_less_opti", "Li6PS5Cl_type5_ver4_a_less_opti", "Li6PS5Cl_type5_ver4_b_less_opti", 
#                                           "Li6PS5Cl_type5_ver1_typ1_less_opti", "Li6PS5Cl_type5_ver1_typ2_less_opti", "Li6PS5Cl_type5_ver2_typ1_less_opti", "Li6PS5Cl_type5_ver2_typ2_less_opti",
#                                           "Li6PS5Cl_type5_ver3_typ1_less_opti", "Li6PS5Cl_type5_ver3_typ2_less_opti", "Li6PS5Cl_type5_ver4_typ1_less_opti", "Li6PS5Cl_type5_ver4_typ2_less_opti"]
file_perfect_poscar_24_wo_cif = "Li6PS5Cl_24_mod_2p27291"

# "Li6PS5Cl_type5_ver1_a", "Li6PS5Cl_type5_ver1_b", "Li6PS5Cl_type5_ver2_a", "Li6PS5Cl_type5_ver2_b", 
#                                           "Li6PS5Cl_type5_ver3_a", "Li6PS5Cl_type5_ver3_b", "Li6PS5Cl_type5_ver4_a", "Li6PS5Cl_type5_ver4_b", 
#                                           "Li6PS5Cl_type5_ver1_typ1", "Li6PS5Cl_type5_ver1_typ2", "Li6PS5Cl_type5_ver2_typ1", "Li6PS5Cl_type5_ver2_typ2",
#                                           "Li6PS5Cl_type5_ver3_typ1", "Li6PS5Cl_type5_ver3_typ2", "Li6PS5Cl_type5_ver4_typ1", "Li6PS5Cl_type5_ver4_typ2",
                                          # "Li6PS5Cl_type5_ver1_a_less", "Li6PS5Cl_type5_ver1_b_less", "Li6PS5Cl_type5_ver2_a_less", "Li6PS5Cl_type5_ver2_b_less", 
                                          # "Li6PS5Cl_type5_ver3_a_less", "Li6PS5Cl_type5_ver3_b_less", "Li6PS5Cl_type5_ver4_a_less", "Li6PS5Cl_type5_ver4_b_less", 
                                          # "Li6PS5Cl_type5_ver1_typ1_less", "Li6PS5Cl_type5_ver1_typ2_less", "Li6PS5Cl_type5_ver2_typ1_less", "Li6PS5Cl_type5_ver2_typ2_less",
                                          # "Li6PS5Cl_type5_ver3_typ1_less", "Li6PS5Cl_type5_ver3_typ2_less", "Li6PS5Cl_type5_ver4_typ1_less", "Li6PS5Cl_type5_ver4_typ2_less"]

# file_ori_ref_48n24 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"

results_folder = "_results"
iter_type = "varying_radius"

litype = 4
activate_radius = 1
max_mapping_radius = 0.075 # 0.025 # 0.043 # 0.035
max_mapping_radius_48htype2 = 0.075 # 0.075 # 0.076 # 0.071 

var_excel_file = "wo_outliars"
max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

folder_name_iter_type = f"/{results_folder}/_{iter_type}/"
path_folder_name_iter_type = direc+str(folder_name_iter_type)

In [78]:
delta_radius = 0.001 
n_sample = 72
var_optitype = "varying_radius"

# radius_sumweirdosLi_df= varying_radius_vs_sumweirdosLi(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, delta_radius, n_sample, var_optitype)
# plot_varying_radius_vs_sumweirdosLi(radius_sumweirdosLi_df)

In [79]:
for file_perfect_poscar_48n24_wo_cif in file_perfect_poscar_48n24_wo_cif_array:
    file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

    col_radius_type1 = "radius_type1"
    col_radius_type2 = "radius_type2"
    col_sumweirdosLi = "sumweirdosLi"

    radius_sumweirdosLi_df = pd.DataFrame()
    radius_sumweirdosLi_df[col_radius_type1] = None
    radius_sumweirdosLi_df[col_radius_type2] = None
    radius_sumweirdosLi_df[col_sumweirdosLi] = None

    idx_sumweirdosLi_df = 0
    # forward_max_mapping_radius_init = max_mapping_radius
    # forward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2
    backward_max_mapping_radius_init = max_mapping_radius
    backward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

    # for i in range(int(n_sample/2)):
    #     forward_max_mapping_radius = forward_max_mapping_radius_init + (i * delta_radius)
    #     # forward_max_mapping_radius_48htype2 = forward_max_mapping_radius_init + (i * delta_radius)
    #     forward_max_mapping_radius_48htype2 = forward_max_mapping_radius_48htype2_init

    #     # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, forward_max_mapping_radius, forward_max_mapping_radius_48htype2, var_optitype)
    #     sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, forward_max_mapping_radius, forward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)

    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type1] = forward_max_mapping_radius
    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type2] = forward_max_mapping_radius_48htype2
    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li
        
    #     idx_sumweirdosLi_df += 1
    #     print(idx_sumweirdosLi_df)

    for j in range(int(n_sample/2)):
        backward_max_mapping_radius = backward_max_mapping_radius_init - (j * delta_radius)
        backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init - (j * delta_radius)
        # backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init
        # backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init

        # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, backward_max_mapping_radius, backward_max_mapping_radius_48htype2, var_optitype)
        # sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, backward_max_mapping_radius, backward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
        sum_weirdos_Li = get_sum_weirdos_Li_var(backward_max_mapping_radius, backward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24_wo_cif, file_perfect_poscar_48n24_wo_cif, litype, var_optitype, iter_type = "varying_radius", foldermapping_namestyle_all = True, cif_namestyle_all = None, modif_all_litype = None)
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type1] = backward_max_mapping_radius
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type2] = backward_max_mapping_radius_48htype2
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        idx_sumweirdosLi_df += 1
        print(idx_sumweirdosLi_df)

    min_sumweirdos = int(radius_sumweirdosLi_df["sumweirdosLi"].min())
    radius_sumweirdosLi_df.to_excel(f'{path_folder_name_iter_type}07_varying_radius_{min_sumweirdos}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)


/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5046: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5050: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

formatted_positions: [-0.02889, 0.25, 0.25, 0.06943, 0.29503, 0.5694, 0.30709, 0.028, 0.69291, 0.33118, 0.02853, 0.66882, 0.78917, 0.00875, 0.21083]


/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

1


/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5046: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5050: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

...

35


/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5046: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:5050: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

formatted_positions: [-0.02889, 0.25, 0.25, 0.06943, 0.29503, 0.5694, 0.30709, 0.028, 0.69291, 0.33118, 0.02853, 0.66882, 0.78917, 0.00875, 0.21083]


/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/01_Masterthesis/01_Precondition/expand_litype/get_weirdos_ver3_new_cleaned.py:668: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

36


In [80]:
# radius_sumweirdosLi_df

In [81]:
# radius_sumweirdosLi_df.plot(x="radius_type1", y="sumweirdosLi")

In [82]:
# min_sumweirdos = int(radius_sumweirdosLi_df["sumweirdosLi"].min())

In [83]:
# path_folder_name_iter_type

In [84]:
# radius_sumweirdosLi_df.to_excel(f'{path_folder_name_iter_type}07_varying_radius_{min_sumweirdos}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)

In [85]:
# radius_sumweirdosLi_df(radius_sumweirdosLi_df)